In [1]:
# library
import pandas as pd

C:\Users\TAMIA\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\TAMIA\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\TAMIA\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data = pd.read_csv('doc/coba.csv')
data

,Waktu,Tweet,Username
0,2021-11-17 22:34:54+00:00,'@idtodayco Polisi sudah bersekutu dgn penjaha...,Sanes Satrio Piningit
1,2021-11-17 18:06:36+00:00,'@Herlin_673YP @DivHumas_Polri @CCICPolri @Muh...,í ë ð ¶ï¸ðï¸ð ±ï¸
2,2021-11-17 17:30:51+00:00,'@txtdrberseragam #1hari1oknum \n#PastiBisa\n#...,POLDA JATENG
3,2021-11-17 17:20:59+00:00,'RT @RINDUNENEKK: Kamu yg sudah lama berharap ...,panggil aja PEN-tol
4,2021-11-17 17:18:57+00:00,'@ksatria_kelam @rio_evi @deddisutrsno @thepat...,StayStrongPalestine ðµð¸ðµð¸ðµð¸
...,...,...,...
1762,2021-11-11 03:00:02+00:00,"'@PolhukamRI Tolong dibantu, sy mendapat SP2HP...",Antonius Marsusuka
1763,2021-11-11 02:53:39+00:00,'@jogipunya Kenapa ga lapor polisi? Eh iya lup...,Name cannot be blank
1764,2021-11-11 01:41:31+00:00,'@ChusnulCh__ @DivHumas_Polri @jokowi @ListyoS...,Kapten thingithink
1765,2021-11-11 01:38:08+00:00,'@her_fnu @TofaTofa_id @is_pelssy @muhammadiya...,Mamiennie


In [32]:
import numpy as np
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize 

In [54]:
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
#   menghilangkan username
    userPattern = '@[^\s]+'
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
#   menghilangkan tautan gambar
    text = re.sub(r'pic.twitter.com.[\w]+', '', text) # Remove every pic 
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    
    text = re.sub(userPattern,' USER', text)  
    text = re.sub(sequencePattern, seqReplacePattern, text)
    
    text = re.sub(r'[^\x00-\xe2]+',' ', text)
    text = re.sub(r':', '', text)
    text = re.sub(r'‚Ä¶', '', text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    
    retweet_tweet = [' rt ']
    retweet_user = [' user ']
    
    for word in retweet_user:
        text = re.sub(word,' ',text) # Remove every retweet symbol & username
        text = re.sub(word.upper(),' ',text)
        
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    text = re.sub('00', ' ', text)
    return text


slang_dict = pd.read_csv('doc/kamus_alay.csv', encoding='latin-1', header=None)
slang_dict = slang_dict.rename(columns={0: 'original', 1: 'replacement'})
slang_dict_map = dict(zip(slang_dict['original'], slang_dict['replacement']))

def normalize_slang(text):
    return ' '.join(map(str, [slang_dict_map[word] if word in slang_dict_map else word for word in text.split(' ')]))

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    return stemmer.stem(text)

id_stopword_dict = pd.read_csv('doc/stopword.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

def remove_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def tokenizing(text):
    return word_tokenize(text)

In [55]:
def preprocess(text):
    text = lowercase(text)
    return text

data_result = data['Tweet'].apply(preprocess)
# data_lower.to_csv('TA 2/datalower.csv')
data_result.head(10)

0    '@idtodayco polisi sudah bersekutu dgn penjaha...
1    '@herlin_673yp @divhumas_polri @ccicpolri @muh...
2    '@txtdrberseragam #1hari1oknum \n#pastibisa\n#...
3    'rt @rindunenekk: kamu yg sudah lama berharap ...
4    '@ksatria_kelam @rio_evi @deddisutrsno @thepat...
5    'rt @viz90: @vivacoid sodara saya sudah lapor ...
6    'rt @tianrostiani: @garutupdate_ @bankbjbuntuk...
7    '@garutupdate_ @bankbjbuntukibu @bankbjbsyaria...
8    '@kumparan jadi begini bapak kombes.....setela...
9    'satu lagi yang mebuktikan bahwa #percumalapor...
Name: Tweet, dtype: object

In [56]:
def preprocess1(text):
    text = remove_unnecessary_char(text)
    text = remove_nonaplhanumeric(text)
    return text

data_cleaning = data_result.apply(preprocess1)
# data_cleaning.to_csv('TA 2/datacleaning.csv')
data_cleaning.head(10)

0     polisi sudah bersekutu dgn penjahat atau mala...
1     gw mah dah bodo amat dah toh ujungnya minta m...
2     harioknum pastibisa percumalaporpolisi nmanya...
3     kamu yg sudah lama berharap tapi hati dia unt...
4     pokoknya asal kata polisi dia salah mrk berge...
5     sodara saya sudah lapor ke propam kasus mobil...
6      percumalaporpolisi nfoto pelaku pencurian uang 
7      percumalaporpolisi nfoto pelaku pencurian uang 
8     jadi begini bapak kombes setelah ketahuan tan...
9     satu lagi yang mebuktikan bahwa percumalaporp...
Name: Tweet, dtype: object

In [57]:
def preprocess2(text):
    text = normalize_slang(text)
    return text

data_normalize = data_cleaning.apply(preprocess2)
# data_normalize.to_csv('TA 2/datanormalize.csv')
data_normalize.head(10)

0     polisi sudah bersekutu dengan penjahat atau m...
1     gue mah deh bodo amat deh toh ujungnya memint...
2     harioknum pastibisa percumalaporpolisi nmanya...
3     kamu yang sudah lama berharap tapi hati dia u...
4     pokoknya asal kata polisi dia salah mereka be...
5     sodara saya sudah lapor ke propam kasus mobil...
6      percumalaporpolisi nfoto pelaku pencurian uang 
7      percumalaporpolisi nfoto pelaku pencurian uang 
8     jadi begini bapak kombes setelah ketahuan tan...
9     satu lagi yang mebuktikan bahwa percumalaporp...
Name: Tweet, dtype: object

In [58]:
def preprocess3(text):
    text = stemming(text)
    return text

data_stemming = data_normalize.apply(preprocess3)
# data_stemming.to_csv('TA 2/datastemming.csv')
data_stemming.head(10)

0    polisi sudah sekutu dengan jahat atau malah su...
1    gue mah deh bodo amat deh toh ujung minta maaf...
2    harioknum pastibisa percumalaporpolisi nmanyal...
3    kamu yang sudah lama harap tapi hati dia untuk...
4    pokok asal kata polisi dia salah mereka gembir...
5    sodara saya sudah lapor ke propam kasus mobil ...
6              percumalaporpolisi nfoto laku curi uang
7              percumalaporpolisi nfoto laku curi uang
8    jadi begini bapak kombes telah tahu tanda like...
9    satu lagi yang mebuktikan bahwa percumalaporpo...
Name: Tweet, dtype: object

In [59]:
def preprocess5(text):
    text = remove_stopword(text)
    return text

data_stopword = data_stemming.apply(preprocess5)
# data_stopword.to_csv('TA 2/datastopword.csv')
data_stopword.head(10)

0    polisi sekutu jahat big bos jahat percumalapor...
1       gue bodo ujung maaf materai percumalaporpolisi
2    harioknum pastibisa percumalaporpolisi nmanyal...
3    harap hati orang ntvri spiderman ulama percuma...
4    pokok polisi salah gembira nikmat tuduh tunggu...
5    sodara lapor propam mobil curi anggota polisi ...
6              percumalaporpolisi nfoto laku curi uang
7              percumalaporpolisi nfoto laku curi uang
8    kombes tanda like unlike admin percumalaporpolisi
9      mebuktikan percumalaporpolisi lucu hukum negeri
Name: Tweet, dtype: object

In [60]:
def preprocess6(text):
    text = tokenizing(text)
    return text

data_tokenizing = data_stopword.apply(preprocess6)
# data_tokenizing.to_csv('datatokenizing.csv')
data_tokenizing.head(10)

0    [polisi, sekutu, jahat, big, bos, jahat, percu...
1    [gue, bodo, ujung, maaf, materai, percumalapor...
2    [harioknum, pastibisa, percumalaporpolisi, nma...
3    [harap, hati, orang, ntvri, spiderman, ulama, ...
4    [pokok, polisi, salah, gembira, nikmat, tuduh,...
5    [sodara, lapor, propam, mobil, curi, anggota, ...
6        [percumalaporpolisi, nfoto, laku, curi, uang]
7        [percumalaporpolisi, nfoto, laku, curi, uang]
8    [kombes, tanda, like, unlike, admin, percumala...
9    [mebuktikan, percumalaporpolisi, lucu, hukum, ...
Name: Tweet, dtype: object

In [61]:
data['Final'] = data_tokenizing
data

,Waktu,Tweet,Username,Final
0,2021-11-17 22:34:54+00:00,'@idtodayco Polisi sudah bersekutu dgn penjaha...,Sanes Satrio Piningit,"[polisi, sekutu, jahat, big, bos, jahat, percu..."
1,2021-11-17 18:06:36+00:00,'@Herlin_673YP @DivHumas_Polri @CCICPolri @Muh...,í ë ð ¶ï¸ðï¸ð ±ï¸,"[gue, bodo, ujung, maaf, materai, percumalapor..."
2,2021-11-17 17:30:51+00:00,'@txtdrberseragam #1hari1oknum \n#PastiBisa\n#...,POLDA JATENG,"[harioknum, pastibisa, percumalaporpolisi, nma..."
3,2021-11-17 17:20:59+00:00,'RT @RINDUNENEKK: Kamu yg sudah lama berharap ...,panggil aja PEN-tol,"[harap, hati, orang, ntvri, spiderman, ulama, ..."
4,2021-11-17 17:18:57+00:00,'@ksatria_kelam @rio_evi @deddisutrsno @thepat...,StayStrongPalestine ðµð¸ðµð¸ðµð¸,"[pokok, polisi, salah, gembira, nikmat, tuduh,..."
...,...,...,...,...
1762,2021-11-11 03:00:02+00:00,"'@PolhukamRI Tolong dibantu, sy mendapat SP2HP...",Antonius Marsusuka,"[tolong, bantu, sphp, kembang, percumalaporpol..."
1763,2021-11-11 02:53:39+00:00,'@jogipunya Kenapa ga lapor polisi? Eh iya lup...,Name cannot be blank,"[lapor, polisi, iya, lupa, percumalaporpolisi]"
1764,2021-11-11 01:41:31+00:00,'@ChusnulCh__ @DivHumas_Polri @jokowi @ListyoS...,Kapten thingithink,"[stop, viral, sop, tunggu, viral, cari, barang..."
1765,2021-11-11 01:38:08+00:00,'@her_fnu @TofaTofa_id @is_pelssy @muhammadiya...,Mamiennie,"[tolong, kapolri, waarga, percumalaporpolisi]"


In [62]:
x = data[['Tweet', 'Final']]

In [63]:
x

,Tweet,Final
0,'@idtodayco Polisi sudah bersekutu dgn penjaha...,"[polisi, sekutu, jahat, big, bos, jahat, percu..."
1,'@Herlin_673YP @DivHumas_Polri @CCICPolri @Muh...,"[gue, bodo, ujung, maaf, materai, percumalapor..."
2,'@txtdrberseragam #1hari1oknum \n#PastiBisa\n#...,"[harioknum, pastibisa, percumalaporpolisi, nma..."
3,'RT @RINDUNENEKK: Kamu yg sudah lama berharap ...,"[harap, hati, orang, ntvri, spiderman, ulama, ..."
4,'@ksatria_kelam @rio_evi @deddisutrsno @thepat...,"[pokok, polisi, salah, gembira, nikmat, tuduh,..."
...,...,...
1762,"'@PolhukamRI Tolong dibantu, sy mendapat SP2HP...","[tolong, bantu, sphp, kembang, percumalaporpol..."
1763,'@jogipunya Kenapa ga lapor polisi? Eh iya lup...,"[lapor, polisi, iya, lupa, percumalaporpolisi]"
1764,'@ChusnulCh__ @DivHumas_Polri @jokowi @ListyoS...,"[stop, viral, sop, tunggu, viral, cari, barang..."
1765,'@her_fnu @TofaTofa_id @is_pelssy @muhammadiya...,"[tolong, kapolri, waarga, percumalaporpolisi]"


In [64]:
x.to_csv('result/hasil_preprocessing.csv')